<a href="https://colab.research.google.com/github/cerfs21/notebooks/blob/main/Olivier_5_AirQuality_Prediction_lstm_exercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deep learning: Time Series forecasting
**Author:** [Dr. Habiboulaye Amadou-Boubacar](https://www.linkedin.com/in/habiboulaye-amadou-boubacar-8b153710)

This notebook takes inspiration from works of Jason Brownlee

## Tutorial on Deep learning: Time Series forecasting
### Recurent Neural Networks: Develop an LSTM (Long Short-Term Memory)
* Air Pollution data
* Baseline model
* Vanilla LSTM model
* LSTM with scaled data
* StackedLSTM
* LTSM-CNN (optional)


### Populating namespace

In [ ]:
import pandas as pd
from matplotlib import pylab as plt
from datetime import datetime
import numpy as np
from math import sqrt
from numpy import concatenate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

### Air Pollution dataset ([Beijing PM2.5 Data Set](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv))
The dataset is collected from the US embassy in Beijing, China. It reports the Air quality and the weather each hour for five(5) years.
The data including the pollutant (PM2.5 concentration) to forecast is described with varibles listed below:
* No: row number
* year: year of data in this row
* month: month of data in this row
* day: day of data in this row
* hour: hour of data in this row
* pm2.5: PM2.5 concentration
* DEWP: Dew Point
* TEMP: Temperature
* PRES: Pressure
* cbwd: Combined wind direction
* Iws: Cumulated wind speed
* Is: Cumulated hours of snow
* Ir: Cumulated hours of rain

The Goal is to forecast the pollution at the next hour given history of pollution and weather condition



#### Download [Beijing PM2.5 Data Set](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv)

In [ ]:
!rm pollution.csv
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv
!ls

<font color='red'>
<b>EXERCICES</b>: Replace the <b>FILL_IN</b> pattern with the correct codes then execute the cell
 </font>

In [ ]:
data = pd.read_csv('pollution.csv')
print(data.shape)
# Display the first (5) rows of the dataframe
<FILL_IN>

In [ ]:
# Check NAs (missing values) for all the columns
<FILL_IN>

In [ ]:
# Check date types
<FILL_IN>

In [ ]:
#Check variable cbwd value occurences using value_counts
<FILL_IN>

#### Reload, preprocess and visualize data
<font color='red'>
EXERCICE: Replace the <FILL_IN> with the correct codes to complete the code
 </font>

In [ ]:
# Load & format date
data = pd.read_csv('pollution.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=lambda d: datetime.strptime(d, '%Y %m %d %H'))
# Drop No(variable) column
<FILL_IN>
# Fill all the NA with value  0
<FILL_IN>
# label encodeing of cbwd (wind direction feature): categories to numerics
<FILL_IN> # Create an instance of LabelEncoder
<FILL_IN> # use the fit__transform function to encode the data
# rename columns
data.rename(columns = {"pm2.5":"pollution","cbwd":"WINDdir","Iws":"WINDsped",	"Is":"SNOW",	"Ir":"RAIN"}, inplace=True)
data.index.name = 'datetime'
data.head()

In [ ]:
plt.figure(figsize=(15,20))
list_vars = data.columns #("pollution",	"DEWP",	"TEMP",	"PRES",	"WINDsped",	"SNOW",	"RAIN")
for i, var in enumerate(list_vars):
  plt.subplot(data.shape[1], 1, i+1)
  plt.plot(data[var], color='green')
  plt.title(var,y=0.8)

In [ ]:
data.pollution.hist(bins=50)

## Time serie forecasting as supervised learning problem (Regression setting)
* predict the pollution at the next hour (t) given the pollution and weather conditions at the prior time step

<font color='red'>
EXERCICE: Replace the <FILL_IN> with the correct codes to complete the code
 </font>

#### Prepare the pollution dataset for LSTM algorithm

In [ ]:
def history_and_horizon_sequencing(df, n_history, n_horizon, target=None):
  # History: look-back sequences (t-n, ... t-1)
  stack_history = []
  for i in range(n_history,0,-1):
    df_i = df.shift(i)
    df_i.columns = [f'{col}_t-{i}' for col in df_i.columns]
    stack_history =  stack_history + [df_i]
  # target dataframe
  if target is None: df_target = df
  else: df_target = df[target].to_frame()
  # Horizon: step-ahead sequences (t+1, ... t+n)
  stack_horizon = []
  for j in range(n_horizon,0,-1):
    df_j = df_target.shift(j)
    df_j.columns = [f'{col}_t+{j}' for col in df_j.columns]
    stack_horizon = [df_j] + stack_horizon
  # Present: t
  df_t = df.copy()
  df_t.columns = [f'{col}_t' for col in df_t.columns]
  # return the concatenated data frame: past+present+future
  return pd.concat(stack_history+[df_t]+stack_horizon, axis=1)

In [ ]:
data_Xy = history_and_horizon_sequencing(data, 3, 1, target='pollution')
print(data_Xy.shape)
data_Xy.dropna(inplace=True)
print(data_Xy.shape)
data_Xy.head()

#### Baseline v0: persistance modeling
Use observation from the present time step (t) to predict the observation at the next time step (t+1).   

In [ ]:
def baseline_persistance(serie_t):
  # forecast: predict t+1 with value of t
  <FILL_IN>

def compute_performance(test, forecast, start_t=400, end_t=500):
  <FILL_IN> # Compute root mean squared error between test and forecast
  print('RMSE: %.3f' % rmse)
  plt.figure(figsize=(15,5))
  plt.plot(test[start_t:end_t], color='b')
  plt.plot(forecast[start_t:end_t], color='r')

In [ ]:
test_y, persistance_yhat = data_Xy['pollution_t'][1:], baseline_persistance(data_Xy['pollution_t'])[1:]
compute_performance(test_y, persistance_yhat, start_t=365*24, end_t=365*24+200)

#### LSTM model

#### Split data into train and test sets
Build train and test dataset for training the model on the first one(1) year and prediction of the four(4) lastest years

In [ ]:
def split_train_test(values, n_train_hours = 365*24):
  # Split the data into train (use values before n_train_hours) and test sets (use values after n_train_hours)
  <FILL_IN>
  <FILL_IN>
  # split into input and outputs
  train_X, train_y = train[:, :-1], train[:, -1]
  test_X, test_y = test[:, :-1], test[:, -1]
  # reshape input to be 3D [samples, timesteps, features] required for LSTM
  train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
  test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
  print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
  return train_X, train_y, test_X, test_y

In [ ]:
train_X, train_y, test_X, test_y = split_train_test(data_Xy.values)

#### Build an LSTM model

In [ ]:
def build_lstm_model(input_shape, nb_neurons = 50):
  '''
  train and test: input shape 1 time step with 8 features
  LSTM:
   * nb_neurons: hidden state
   * 1 neuron for output layer for prediction.
  '''
  model = Sequential()
  # Add an LSTM layer with nb_neurons and input_shape=input_shape
  <FILL_IN>
  # Add a Dense layer with one output neuron
  <FILL_IN>
  # model compile
  model.compile(loss='mae', optimizer='adam')
  # Train the model
  return model

def train_model(model, train_X, train_y, test_X, test_y, epochs=500, batch_size=72, verbose=0):
  '''
  Generic function to train model
  '''
  history = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_X, test_y), verbose=verbose, shuffle=False)
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.legend()
  plt.show()

In [ ]:
# Build and train the model
input_shape=(train_X.shape[1], train_X.shape[2])
# Build and train the model using the build_lstm_model (new instance lstm_model)
<FILL_IN>
# Train the new model using train_model method with params lstm_model, train_X, train_y, test_X, test_y, epochs=100, batch_size=72, verbose=0
<FILL_IN>

In [ ]:
# make the prediction of test_X (store in lstm_yhat)
lstm_yhat = lstm_model.predict(test_X)
# compute performance
compute_performance(test_y, lstm_yhat, start_t=365*24, end_t=365*24+200)

#### LSTM trained with scaling data

In [ ]:
# Normalize features
values = data_Xy.values.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(values)
# Create train/tests datasets from scaled_data
scaled_train_X, scaled_train_y, scaled_test_X, scaled_test_y = split_train_test(scaled_values)

In [ ]:
# Build and train the model using the build_lstm_model (new instance name lstm_model_scaled)
<FILL_IN>
# Train the new model using train_model method as previousely using the new scaled data
<FILL_IN>

In [ ]:
# invert scaling for forecast
def invert_scaling(scaled_X, scaled_yhat):
  scaled_tX = scaled_test_X.reshape((scaled_X.shape[0], scaled_X.shape[2]))
  Xyat = concatenate((scaled_tX, scaled_yhat.reshape(-1,1)), axis=1)
  inv_Xyat = scaler.inverse_transform(Xyat)
  return inv_Xyat[:,-1]

# make a prediction
lstm_scaled_yhat = lstm_model_scaled.predict(scaled_test_X)
# compute performance
inv_yhat = invert_scaling(scaled_test_X, lstm_scaled_yhat)
compute_performance(test_y, inv_yhat, start_t=365*24, end_t=365*24+200)

#### Build an Stacked LSTM model

Stacked LSTM model refers to stacking multiple hidden LSTM layers one on top of another.

In [ ]:
def build_stacked_lstm_model(input_shape, nb_neurons = 50):
  model = Sequential()
  # Add an LSTM layer with nb_neurons and input_shape=input_shape and return_sequences=True (required sequences needed for next LSTM Bloc)
  <FILL_IN>
  # Add an additional LSTM layer with nb_neurons and 'relu' activation function
  <FILL_IN>
  # Add a Dense layer with one output neuron
  <FILL_IN>
  # Train the model
  return model

In [ ]:
input_shape = (train_X.shape[1], train_X.shape[2])
# Build the new stacked_lstm_model
<FILL_IN>
# Train the model stacked_lstm_model
<FILL_IN>

In [ ]:
# make a prediction with stacked_lstm_model using scaled_test_X (output in variable scaled_stacked_lstm_yhat)
<FILL_IN>
# compute performance
inv_yhat = invert_scaling(scaled_test_X, scaled_stacked_lstm_yhat)
compute_performance(test_y, inv_yhat, start_t=365*24, end_t=365*24+200)

## Assignment: Work to improve the performance - try different strategies:
* Hyperparameters tuning
* Test other models:
 - Eg. 1D-ConvNet LSTM, Bidirectional